# Parse SD monthly voter registration report

Uses:
- `requests` to get the PDF from [the Secretary of State's website](https://sdsos.gov/elections-voting/upcoming-elections/voter-registration-totals/voter-registration-by-county.aspx)
- `pdfplumber` to parse and validate the data

In [1]:
from io import BytesIO
from datetime import datetime
import csv

import requests
import pdfplumber

In [2]:
url = 'https://sdsos.gov/elections-voting/assets/StatewideVoterRegistrationTotals/StatewideVotersByCounty_8.1.2024.pdf'
req = requests.get(url)

In [3]:
pdf = pdfplumber.open(BytesIO(req.content))

In [4]:
# assumes a 2-page PDF with a table spanning both pages
if len(pdf.pages) != 2:
    raise Exception(f'Report page count is {len(pdf.pages)}, not 2 -- check it out: {url}')

In [5]:
lines = pdf.pages[0].extract_table() + pdf.pages[1].extract_table()

# guarding against some PDFs parsed with an extra column
lines = [x[:-1] if not x[-1] else x for x in lines]

In [6]:
lines

[['Statewide Report by County - August 1, 2024',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 ['County',
  'Democratic',
  'Libertarian',
  'Republican',
  'Independent',
  'OTH',
  'No Party\nAffiliation',
  'No Labels',
  'Inactive',
  'Total Active'],
 ['Aurora', '524', '2', '970', '222', '2', '96', '', '111', '1816'],
 ['Beadle',
  '2,540',
  '26',
  '4,972',
  '1,398',
  '40',
  '1,083',
  '1',
  '710',
  '10060'],
 ['Bennett', '723', '6', '727', '220', '2', '198', '', '122', '1876'],
 ['Bon Homme', '894', '12', '2,339', '475', '', '209', '', '299', '3929'],
 ['Brookings',
  '4,970',
  '101',
  '9,584',
  '3,548',
  '21',
  '2,607',
  '',
  '1620',
  '20831'],
 ['Brown',
  '7,269',
  '96',
  '12,561',
  '3,554',
  '11',
  '1,776',
  '1',
  '1150',
  '25268'],
 ['Brule', '904', '14', '1,697', '458', '4', '200', '', '177', '3277'],
 ['Buffalo', '530', '2', '151', '67', '3', '80', '', '123', '833'],
 ['Butte', '774', '56', '4,494', '885', '20', '605', '', '1353',

In [7]:
# report date is in the first line
report_date = lines.pop(0)

In [8]:
report_date = report_date[0].split('-')[-1].strip()
report_date = datetime.strptime(report_date, '%B %d, %Y').date().isoformat()

print(report_date)

2024-08-01


In [9]:
# standardizing column names
column_fixes = {
    'oth': 'other',
    'no party\naffiliation': 'npa',
    'no labels': 'no_labels',
    'npa/ind': 'npa_ind'
}

In [10]:
# headers are in the first line of the data table
headers = lines.pop(0)
headers = [column_fixes.get(x.lower(), x.lower()) for x in headers]

In [11]:
headers

['county',
 'democratic',
 'libertarian',
 'republican',
 'independent',
 'other',
 'npa',
 'no_labels',
 'inactive',
 'total active']

In [12]:
data = []

# loop over the remaining lines
for line in lines:

    first_cell = line[0].lower()

    # skip lines if repeated headers
    if 'statewide' in first_cell or 'county' in first_cell:
        continue

    # the 'Total' line in the table means we're done
    if 'total' in first_cell:
        break

    # check that the number of values matches the number of headers
    assert(len(line) == len(headers))

    # zip it up into a dict
    county_data = dict(zip(headers, line))

    # delete the aggregate value totaling everything up
    del county_data['total active']

    # cast number strings to integers
    for key in county_data:
        if key == 'county' or not county_data[key]:
            continue

        county_data[key] = int(county_data[key].replace(',', ''))

    # add the report date
    county_data['date'] = report_date
    data.append(county_data)

In [13]:
# doublecheck: 66 counties in SD
assert(len(data) == 66)

In [14]:
data[0]

{'county': 'Aurora',
 'democratic': 524,
 'libertarian': 2,
 'republican': 970,
 'independent': 222,
 'other': 2,
 'npa': 96,
 'no_labels': '',
 'inactive': 111,
 'date': '2024-08-01'}

In [15]:
# shift 'date' to the beginning of headers list for the output CSV
csv_headers = list(data[0].keys())
csv_headers = csv_headers[-1:] + csv_headers[:-1]

In [16]:
# write to file
filepath = f'data/{report_date}.csv'

with open(filepath, 'w') as outfile:
    writer = csv.DictWriter(
        outfile,
        fieldnames=csv_headers
    )

    writer.writeheader()
    writer.writerows(data)

    print(f'Wrote file: {filepath}')

Wrote file: data/2024-08-01.csv
